In [1]:
import math

from pathlib import Path
import shutil
from typing import List
import numpy as np

from SURE.custom.imageModeConversion import convert_images_in_directory

handpicked_number = 2

convert_images_in_directory(f'SURE/datasets/handpicked/{handpicked_number}/pre_augmented/original', f'SURE/datasets/handpicked/{handpicked_number}/pre_augmented/images', target_mode="RGB")

Processing 1.png: Original mode: L
Converted mode: RGB
Converted image saved to: SURE/datasets/handpicked/2/pre_augmented/images\1.png
Processing 32.png: Original mode: RGBA
Converted mode: RGB
Converted image saved to: SURE/datasets/handpicked/2/pre_augmented/images\32.png
Processing 33.png: Original mode: RGBA
Converted mode: RGB
Converted image saved to: SURE/datasets/handpicked/2/pre_augmented/images\33.png
Processing 46.png: Original mode: I
Converted mode: RGB
Converted image saved to: SURE/datasets/handpicked/2/pre_augmented/images\46.png
Processing 47.png: Original mode: I
Converted mode: RGB
Converted image saved to: SURE/datasets/handpicked/2/pre_augmented/images\47.png
Processing 48.png: Original mode: I
Converted mode: RGB
Converted image saved to: SURE/datasets/handpicked/2/pre_augmented/images\48.png
Processing 50.png: Original mode: I
Converted mode: RGB
Converted image saved to: SURE/datasets/handpicked/2/pre_augmented/images\50.png
Processing 52.png: Original mode: I
C

In [2]:
def _CopyToPath(paths: List[Path], output: Path):
    output.mkdir(parents=True, exist_ok=True)
    for path in paths:
        newPath = output / path.name
        shutil.copy(path, newPath)

def SplitData(imagePaths: List[Path], segmentationPaths: List[Path], validationFraction: float,
              testingFraction: float,
              outputDirectory: Path,
              required_numbers: List[int]):
    # Sort paths alphabetically
    imagePaths.sort(key=lambda x: x.stem)
    segmentationPaths.sort(key=lambda x: x.stem)

    # Only use paths that have matched names in image and segmentations directory
    imagePaths = np.asarray([path for path in imagePaths if
                             path.stem in [segPath.stem for segPath in segmentationPaths]])
    segmentationPaths = np.asarray([path for path in segmentationPaths if
                                    path.stem in [imagePath.stem for imagePath in imagePaths]])

    # Carry out the split!
    permutation = np.random.permutation(len(imagePaths))
    
    for n in required_numbers:
        adjusted_number = math.ceil(n/(1-validationFraction-testingFraction))
        current_portion = permutation[:adjusted_number]
        numValidation = math.ceil(adjusted_number * validationFraction)
        
        trainingIndices = current_portion[:n]
        validationIndices = current_portion[n:(n + numValidation)]
        testingIndices = current_portion[(n + numValidation):adjusted_number]

        # Save the images
        _CopyToPath(imagePaths[trainingIndices], outputDirectory / str(n) / "training" / "images")
        _CopyToPath(imagePaths[validationIndices], outputDirectory / str(n) / "validation" / "images")
        _CopyToPath(imagePaths[testingIndices], outputDirectory / str(n) / "testing" / "images")
        _CopyToPath(segmentationPaths[trainingIndices], outputDirectory / str(n) / "training" / "segmentations")
        _CopyToPath(segmentationPaths[validationIndices],
                    outputDirectory / str(n) / "validation" / "segmentations")
        _CopyToPath(segmentationPaths[testingIndices], outputDirectory / str(n) / "testing" / "segmentations")

In [ ]:
images_dir = Path('Resources/Images')
segmentations_dir = Path('Resources/Segmentation')
output_dir = Path('Oversampling')

count = 1000

AugmentImages(imagesDirectory=images_dir, segmentationsDirectory=segmentations_dir, outputDirectory=output_dir, count=count)

image_paths = list(images_dir.glob('*.png'))  
segmentation_paths = list(segmentations_dir.glob('*.png')) 

In [ ]:
SplitData()